In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [43]:
import pandas as pd
df= pd.read_csv('PLCdata.csv', index_col=[0])

In [44]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1398 entries, 0 to 1397
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Fault              1398 non-null   object
 1   Diagnostic         1398 non-null   object
 2   Corrective Action  1397 non-null   object
 3   PLC                1398 non-null   object
 4   Model              1398 non-null   object
dtypes: object(5)
memory usage: 65.5+ KB
None


,Fault,Diagnostic,Corrective Action,PLC,Model
0,cracked glass failure,broken or cracked glass electrode,replace electrode if cracked,emerson,model e
1,cracked glass failure,broken or cracked glass electrode,check wiring for short,emerson,model e
2,zero offset error,reference electrode poisoned,replace reference electorode,emerson,model e
3,high reference imped,coated reference electrode,clean electrode as instructed in sensor manual,emerson,model e
4,high reference imped,sensor out of process,clean electrode as instructed in sensor manual,emerson,model e


In [45]:
df

,Fault,Diagnostic,Corrective Action,PLC,Model
0,cracked glass failure,broken or cracked glass electrode,replace electrode if cracked,emerson,model e
1,cracked glass failure,broken or cracked glass electrode,check wiring for short,emerson,model e
2,zero offset error,reference electrode poisoned,replace reference electorode,emerson,model e
3,high reference imped,coated reference electrode,clean electrode as instructed in sensor manual,emerson,model e
4,high reference imped,sensor out of process,clean electrode as instructed in sensor manual,emerson,model e
...,...,...,...,...,...
1393,bolock input error,insufficient memory space cannot add a block t...,increase the memory space,siemens,logo
1394,wiring problem,physical cable connections from the end device...,use the proper physical cable connections from...,siemens,logo
1395,wiring problem,there is no appropriate conductor crosssection...,make sure you use cables with appropriate cond...,siemens,logo
1396,wiring problem,cable length exceeds the specifications,make sure the cable length does not exceed spe...,siemens,logo


In [48]:
df.dropna(inplace=True)

In [49]:
import spacy

# Load the English model
nlp = spacy.load("en_core_web_sm")

# Define a function to lemmatize the text
def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

# Apply the lemmatization function to each row of the dataframe
df["Diagnostic"] = df["Diagnostic"].apply(lemmatize_text)
df["Corrective Action"] = df["Corrective Action"].apply(lemmatize_text)
df["Fault"] = df["Fault"].apply(lemmatize_text)
df["PLC"] = df["PLC"].apply(lemmatize_text)
df["Model"] = df["Model"].apply(lemmatize_text)

In [52]:
df.sample(8)

,Fault,Diagnostic,Corrective Action,PLC,Model
234,not repairable internal system error,the link could not be insert link be distort i...,use the function save as together with the opt...,siemen,step
1271,badpassword,the password define for the remote access serv...,define the password again both on the provider...,abb,xa
858,packet capture problem,packet from the device be analyze be not captu...,to allow the packet capture tool to see the pa...,schneider,modicon tsx
306,the internet explorer be not instal in your pc,the online help be start and the internet expl...,reinstall step,siemen,step
113,detail d can not load sdb xx,h connection be configure in your project the ...,in the simatic manager delete the system datum...,siemen,step
190,the distribute io system datum block be struct...,a diagnostic address outside the address area ...,change the diagnostic address,siemen,step
1120,information of security level timeup,the security level have not be revert,revert the security level to,yokogawa,centum
837,a device can be discover but can not be access...,incorrect community string,the read string must be correct to read datum ...,schneider,modicon tsx


In [63]:
from google.colab import files
df.to_csv("PLC_dataset_new.csv")
files.download("PLC_dataset_new.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [77]:
print(df["PLC"].nunique())
print(df["Model"].nunique())

12
18


In [81]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Initialize the bag-of-words vectorizer
vectorizer = CountVectorizer()

# Fit the vectorizer to the text data and transform it into a bag-of-words representation
X_text = vectorizer.fit_transform(df["Fault"])

# Create the column transformer
ct = ColumnTransformer(
    transformers=[
        ("num", OneHotEncoder(), ["PLC", "Model"])
    ])
# Fit the column transformer to the data and transform it
X_numerical = ct.fit_transform(df[["PLC", "Model"]])
n_encoders = X_numerical.shape[1]

# Combine the text and numerical data into a single input array
X = np.hstack([X_text.toarray(), X_numerical.reshape(-1, n_encoders)])

# Prepare the output data
y = df[["Diagnostic", "Corrective Action"]]



ValueError: ignored

In [53]:
!pip install streamlit --quiet
! pip install pyngrok --quiet

In [62]:
%%writefile app.py
import streamlit as st
st.write("# PLC Fault Prediction")
st.text_input("Enter the PLC:")
st.text_input("Enter the Model:")
st.text_input("Enter the paper Fault:")

Overwriting app.py


In [58]:
from pyngrok import ngrok

ngrok.set_auth_token("2LArVCZnMmuPMV0uYjEx4zAOSpx_6AiMAuvnRpVMQxFzSkvaE") 

!nohup streamlit run app.py --server.port 80 & --quiet
url = ngrok.connect(port = '80')
print(url)

/bin/bash: --quiet: command not found
nohup: appending output to 'nohup.out'
NgrokTunnel: "http://6b6c-34-83-144-170.ngrok.io" -> "http://localhost:80"


In [82]:
ngrok.kill()